<a href="https://colab.research.google.com/github/GaelleNovales/Trendr/blob/master/Tweetsearchipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Ressources

##Téléchargement des ressources



In [9]:
!pip install nltk
!pip install twint
!pip install the Twython library 
!pip install emoji

import nltk
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
nltk.download('sentiwordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package sentiwordnet to /root/nltk_data...
[nltk_data]   Package sentiwordnet is already up-to-date!


True

##Imports

In [0]:
import twint
from datetime import datetime
import emoji
import regex
import re
from nltk import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk import WordNetLemmatizer
import pandas as pd
from textblob import TextBlob
import csv
from datetime import date, datetime
from collections import Counter
import json

#Création du projet

## Importation des tweets selon un hashtag


In [0]:
def export_tweets(mention,taille):
  date = datetime.now()
  date_format = str(date.year)+'-'+str(date.month)+'-'+str(date.day)

  # Configure
  c = twint.Config()
  c.Search = mention
  c.Store_csv = True
  c.Output = c.Search+"_saved.csv"
  c.Limit = taille
  c.Since = date_format
  c.Lang = 'en'
  c.Pandas = True

  # Run
  twint.run.Search(c)

  twint_data = twint.storage.panda.Tweets_df
  return twint_data

##Extraction des emojis

In [0]:
def extract_emojis(a_list):
    emojis_list = map(lambda x: ''.join(x.split()), emoji.UNICODE_EMOJI.keys())
    r = re.compile('|'.join(re.escape(p) for p in emojis_list))
    aux=[' '.join(r.findall(s)) for s in a_list]
    return(aux)

##Traduire les abréviations

In [0]:
def translator(user_string):
    user_string = user_string.split(" ")
    j = 0
    for _str in user_string:
        # File path which consists of Abbreviations.
        fileName = "slang.txt"
        # File Access mode [Read Mode]
        accessMode = "r"
        with open(fileName, accessMode) as myCSVfile:
            # Reading file as CSV with delimiter as "=", so that abbreviation are stored in row[0] and phrases in row[1]
            dataFromFile = csv.reader(myCSVfile, delimiter="=")
            # Removing Special Characters.
            _str = re.sub('[^a-zA-Z0-9-_.]', '', _str)
            for row in dataFromFile:
                # Check if selected word matches short forms[LHS] in text file.
                if _str.upper() == row[0]:
                    # If match found replace it with its appropriate phrase in text file.
                    user_string[j] = row[1]
            myCSVfile.close()
        j = j + 1
    # Replacing commas with spaces for final output.
    return (' '.join(user_string))

##Cleaning du texte

In [0]:
def clean_text(text,mention):
    """
    This function takes as input a text on which several 
    NLTK algorithms will be applied in order to preprocess it
    """
    # Remplacer les abréviations
    #print(text)
    #print(translator(text))
    text = translator(text)

    tokens = word_tokenize(text)
      
    # Remove the punctuations
    tokens = [word for word in tokens if word.isalpha()]
    # Lower the tokens
    tokens = [word.lower() for word in tokens]

    # Remove stopword
    tokens = [word for word in tokens if not word in stopwords.words("english")]

    # Corriger les fautes
    #for word in tokens:
    #  print(word)
    #  print(str(TextBlob(word).correct()))
    tokens = [str(TextBlob(word).correct()) for word in tokens if word != mention]

    # Lemmatize
    lemma = WordNetLemmatizer()
    tokens = [lemma.lemmatize(word, pos = "v") for word in tokens]
    tokens = [lemma.lemmatize(word, pos = "n") for word in tokens]
    return tokens

#Execution du programme

In [0]:
def prog(mention,taille):
  Tweets_df = export_tweets(mention,taille)

  tweet_cleaned = []
  for i in range(len(Tweets_df)):
    tweet_cleaned.append(clean_text(Tweets_df['tweet'][i],mention))  

  emojisExtracted = extract_emojis(Tweets_df['tweet'])
  Tweets_df["tweet_emojis"] = emojisExtracted
  Tweets_df["tweet_cleaned"] = tweet_cleaned
  return Tweets_df

In [230]:
Tweets_df = prog("#pepsi",100)
#df.drop('column_name', axis=1, inplace=True)

1244634481367093248 2020-03-30 14:35:59 UTC <EbbaQ> Love it when the big giants come together to help the entire nation. Thank you #Pepsi !  https://twitter.com/pepsipakistan/status/1244611497403338752 …
1244631321671921670 2020-03-30 14:23:25 UTC <warisha_rais> What a brilliant initiative and I hope more details would be unveiled soon so we all can play our part in supporting our people. #pepsi  https://twitter.com/pepsipakistan/status/1244611497403338752 …
1244617488245145601 2020-03-30 13:28:27 UTC <doberhoocom> #pepsi SlowGuns - The Dober Hoo Stuff  Thousand pages for amazing book  pic.twitter.com/0Fexu6DQT9
1244609224195399683 2020-03-30 12:55:37 UTC <carbonateddave> Pepsi drinkers are the queen of the playground! #pepsi
1244609192943669248 2020-03-30 12:55:29 UTC <selzyboy> @wizkidayo as I like you reach. You have not reached out to us #wizkidfc on how you’re helping. Give us money to go and buy #PEPSI @pepsi naaa  I don’t know sha. Some music would heal us
1244596620362039296 20

#Exportation en JSON

## On réctifie les dates

In [0]:
Tweets_df['date_jour']=datetime.strptime(Tweets_df['date'][0][:10], ('%Y-%m-%d')).strftime("%A")
Tweets_df['date']=Tweets_df['date'][0][:10]

## Pour les nuages de mots

In [0]:
listeH = []
for i in Tweets_df['hashtags'].values:
  listeH = listeH+i
listeH = Counter(listeH).most_common(50)

listeM = []
for i in Tweets_df['tweet_cleaned'].values:
  listeM = listeM+i
listeM = Counter(listeM).most_common(50)

In [211]:
Tweets_df.groupby('date_jour').mean().nlikes.values

,created_at,user_id,day,retweet,nlikes,nreplies,nretweets
date_jour,,,,,,,
Monday,1.585553e+12,6.009514e+17,3.882353,False,1.647059,0.294118,0.117647


## On exporte le tout en JSON

In [0]:
data = {}
data['pepsi'] = []
data['pepsi'].append({
    'nbPositif': '',
    'nbNegatif': '',
    'moyenne': '',
    'date': Tweets_df.date.unique().tolist(),
    'jour': Tweets_df.date_jour.unique().tolist(),
    'nbLikePos': '',
    'nbRetweetPos': '',
    'nbLikeNeg': '',
    'nbRetweetNeg': '',
    'hastagAssocie': listeH,
    'motClefPos': listeM,
    'motClefNeg': listeM
})

with open('data.json', 'w') as outfile:
    json.dump(data, outfile)

#Idées

##Correction de sentences

In [0]:
from textblob import TextBlob

b = TextBlob("I havv goood speling!")
print(b.correct())

I have good spelling!


##Tokénisation

In [0]:
nltk.pos_tag(['go'])

[('go', 'VB')]

##Emotions *par mot*

In [0]:
from nltk.corpus import sentiwordnet as swn
list(swn.senti_synsets('love'))

breakdown = swn.senti_synset('kill.v.01')
print("pos_score: " , breakdown.pos_score())
print("neg_score: " , breakdown.neg_score())
print("obj_score: " , breakdown.obj_score())


pos_score:  0.0
neg_score:  0.5
obj_score:  0.5


##2eme essai de cleaning

In [0]:
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer 

#create an object of class PorterStemmer
stop_words = set(stopwords.words('english'))   

for i in range(len(Tweets_df)):
  word_tokens = word_tokenize(Tweets_df['tweet'][i])   
  filtered_sentence = [w for w in word_tokens if not w in stop_words] 
  filtered_sentence = []   
  for w in word_tokens: 
      if w not in stop_words: 
          filtered_sentence.append(w) 
  #print(word_tokens) 
  #print(filtered_sentence)
  tweet_rmStop.append(filtered_sentence)
  
tweet_lemm = tweet_rmStop

In [0]:
# import these modules 
from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer

#create an object of class PorterStemmer
porter = PorterStemmer()
lancaster=LancasterStemmer()

for i in range(len(tweet_rmStop)):
  for j in range(len(tweet_rmStop[i])):
    tweet_lemm[i][j] = porter.stem(tweet_rmStop[i][j])
    print(tweet_lemm[i][j])

In [0]:
import pandas
#Tweets_df = pandas.DataFrame()

Tweets_df["tweet_rmStop"]= tweet_rmStop
Tweets_df["tweet_lemm"] = tweet_lemm
Tweets_df["tweet_emojis"] = emojisExtracted